# QRC-Lab: Introduction to Quantum Reservoir Computing

Welcome to **QRC-Lab**, an educational framework for simulating **Quantum Reservoir Computing (QRC)**. 

### What is QRC?
Reservoir Computing is a machine learning paradigm where an input signal is mapped into a high-dimensional "reservoir" that evolves dynamically. In QRC, this reservoir is a quantum system (qubits). The key idea is that we **do not train** the quantum reservoir; we only train a simple classical linear model (the Readout) to extract information from the quantum state.

This notebook guide you through the 4 essential steps:
1. **Encoding**: How to turn classical data into quantum states.
2. **Reservoir Dynamics**: The fixed quantum "brain" that mixes information.
3. **Simulation**: Running the time-series loop.
4. **Reading**: Training a model to solve a task.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Core QRC-Lab modules
from qrc_sim.encoders import AngleEncoder, ReuploadingEncoder
from qrc_sim.reservoirs import RandomReservoir, RandomCRotReservoir
from qrc_sim.simulator import QRCSimulator
from qrc_sim.tasks.memory import MemoryTask
from qrc_sim.tasks.narma import NARMADataset
from qrc_sim.readout import ReadoutModel

## Example 1: Short-Term Memory (The Foundation)

A fundamental property of any reservoir is its **memory**. Can the current quantum state "remember" what the input was 1 or 2 steps ago? 

We start by generating a simple signal where the target is just the input delayed by 1 step.

In [ ]:
task = MemoryTask(length=300, delay=1)
(X_train, y_train), (X_test, y_test) = task.generate()

plt.figure(figsize=(12, 3))
plt.plot(X_train[:50], 'o-', label='Input $x_t$', alpha=0.6)
plt.plot(y_train[:50], 's--', label='Target $y_t = x_{t-1}$', alpha=0.8)
plt.title("Short-Term Memory Task Data")
plt.legend()
plt.show()

### Step 2: Configure the Quantum "Brain"

We will use a **ReuploadingEncoder**. Unlike simple angle encoding, this "re-injects" the data multiple times, creating a richer feature space. 
For the reservoir, we use the **RandomCRotReservoir**, which uses randomized controlled rotations for entanglement.

In [ ]:
n_qubits = 4

# Encoder: Injects x_t into the circuit
encoder = ReuploadingEncoder(n_qubits, layers=2)

# Reservoir: The complex, non-linear mixing layer
reservoir = RandomCRotReservoir(n_qubits, depth=2)

# Observables: We will measure the Z-expectation value of every qubit
obs_list = [('Z', i) for i in range(n_qubits)]

# Simulator: Using 'reupload_k' mode to maintain a stable memory window
sim = QRCSimulator(encoder, reservoir, obs_list, state_update_mode='reupload_k', reupload_k=3)

### Step 3: Run and Train

We process the sequence through the quantum simulator to get "Quantum Features", then use a classical **Ridge Regression** to learn the mapping.

In [ ]:
print("Extracting features...")
f_train = sim.run_sequence(X_train)
f_test = sim.run_sequence(X_test)

readout = ReadoutModel(alpha=1e-5)
readout.fit(f_train, y_train)

score = readout.score(f_test, y_test)
print(f"Test R2 Score: {score:.4f}")

In [ ]:
preds = readout.predict(f_test)

plt.figure(figsize=(12, 4))
plt.plot(y_test[:100], label='Target (Truth)', lw=2)
plt.plot(preds[:100], '--', label='QRC Prediction', lw=2)
plt.title(f"Real-time Memory Tracking (R2: {score:.3f})")
plt.legend()
plt.show()

## Example 2: Complex Dynamics (NARMA10)

Now let's try a much harder task: **NARMA10**. It's a standard benchmark in dynamical systems. 
It requires both **long memory** (10 steps) and **high non-linearity**.

In [ ]:
narma = NARMADataset(length=1000, order=10)
(X_n_train, y_n_train), (X_n_test, y_n_test) = narma.generate()

# Using more qubits for the complex task
n_q_large = 8
enc_n = ReuploadingEncoder(n_q_large, layers=2)
res_n = RandomCRotReservoir(n_q_large, depth=2)
obs_n = [('Z', i) for i in range(n_q_large)]

# Large reupload_k to match NARMA order
sim_n = QRCSimulator(enc_n, res_n, obs_n, state_update_mode='reupload_k', reupload_k=11)

f_n_train = sim_n.run_sequence(X_n_train)
f_n_test = sim_n.run_sequence(X_n_test)

readout_n = ReadoutModel(alpha=1e-6)
readout_n.fit(f_n_train, y_n_train)

n_score = readout_n.score(f_n_test, y_n_test)
print(f"NARMA10 Test R2: {n_score:.4f}")

In [ ]:
preds_n = readout_n.predict(f_n_test)

plt.figure(figsize=(12, 4))
plt.plot(y_n_test[:100], label='NARMA Target')
plt.plot(preds_n[:100], '--', label='QRC Prediction')
plt.title(f"NARMA10 Prediction Results")
plt.legend()
plt.show()